<a href="https://colab.research.google.com/github/andramarac/melbourne-real-estate/blob/main/Melbourne's_Real_Estate_Agencies_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Melbourne's Real Estate Agencies Performance** 🏠

Analysis made on: **11th of February 2023** by Andra Maraciuc

**About this dataset**

This analysis is based on data on Property Sales in Melbourne City, from 2016 to 2017, and it covers an in-depth look at the performance of the real estate agencies in that region.

Dataset used (where to download): [Kaggle](https://www.kaggle.com/datasets/amalab182/property-salesmelbourne-city/data)


**Highlights to Analyze**
- Which agencies performed best in terms of sales volume and number of properties sold?
- Which agency achieved the highest sale price for a property?
- What were the monthly sales trends?
- Distribution of property prices across different ranges?
- What are the median and average prices of properties sold by the top-performing agency?
- How does the growth rate of each agency vary year by year?

Tableau Visualization: [View Dashboard](https://public.tableau.com/views/Melbourne-Real-Estate-Agencies-Performance/Dashboard1?:language=en-GB&publish=yes&:display_count=n&:origin=viz_share_link)


## **Data Cleaning and Preparation**

1. **Import packages**

In [ ]:
# load packages
from matplotlib import pyplot as plt
import pandas as pd
import plotly.express as px

# display chart correctly
%matplotlib inline

2. **Load data**

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/funda/Property Sales of Melbourne City.csv")

3. **Sneak peak**

With our data loaded, let's take a peek at how are data looks like.

In [ ]:
# getting the statistical summary of dataset and find outliers
df.describe()

,Unnamed: 0,Rooms,Price,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
count,"18,396","18,396","18,396","18,395","18,395","14,927","14,925","14,820","13,603","7,762","8,958","15,064","15,064","18,395"
mean,"11,827",3,"1,056,697",10,"3,107",3,2,2,558,151,"1,966",-38,145,"7,518"
std,"6,801",1,"641,922",6,95,1,1,1,"3,987",519,37,0,0,"4,488"
min,1,1,"85,000",0,"3,000",0,0,0,0,0,"1,196",-38,144,249
25%,"5,937",2,"633,000",6,"3,046",2,1,1,176,93,"1,950",-38,145,"4,294"
50%,"11,820",3,"880,000",10,"3,085",3,1,2,440,126,"1,970",-38,145,"6,567"
75%,"17,734",3,"1,302,000",13,"3,149",3,2,2,651,174,"2,000",-38,145,"10,331"
max,"23,546",12,"9,000,000",48,"3,978",20,8,10,"433,014","44,515","2,018",-37,146,"21,650"


In [ ]:
# check for missing values
df.isnull().sum()

Unnamed: 0           0
Suburb               0
Address              0
Rooms                0
Type                 0
Price                0
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom2          3469
Bathroom          3471
Car               3576
Landsize          4793
BuildingArea     10634
YearBuilt         9438
CouncilArea       6163
Lattitude         3332
Longtitude        3332
Regionname           1
Propertycount        1
dtype: int64

In [ ]:
# check at the shape of the dataset
df.shape

(18396, 22)

In [ ]:
# check data types
df.dtypes

Unnamed: 0         int64
Suburb            object
Address           object
Rooms              int64
Type              object
Price              int64
Method            object
SellerG           object
Date              object
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Car              float64
Landsize         float64
BuildingArea     float64
YearBuilt        float64
CouncilArea       object
Lattitude        float64
Longtitude       float64
Regionname        object
Propertycount    float64
dtype: object

4. **Delete unnecesary columns**

In [ ]:
df = df.drop(columns=['Unnamed: 0', 'Lattitude', 'Longtitude', 'CouncilArea', 'Propertycount', 'Car','Landsize','BuildingArea','Bathroom','Bedroom2','Propertycount','YearBuilt'])

5. **Data types and conversion**

In [ ]:
# change date to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [ ]:
# change price column to float
df['Price'] = df['Price'].astype(int)

# add comma to floats
pd.options.display.float_format = '{:,.0f}'.format

6. **Rename columns**

In [ ]:
df.rename(columns={'SellerG':'Agency','Date':'SellDate','Bedroom2':'Bedroom','Method':'SaleMethod','Car':'CarSpace','Distance':'DistanceToCenter'},inplace=True)

In [ ]:
# split the date columns to new columns
df['SellDay'] = df['SellDate'].dt.day
df['SellMonth'] = df['SellDate'].dt.month
df['SellYear'] = df['SellDate'].dt.year


In [ ]:
df['SellYear'].astype(int)

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
18391    2017
18392    2017
18393    2017
18394    2017
18395    2017
Name: SellYear, Length: 18396, dtype: int64

7. **Fix the missing values**


As we will might use the *DistanceToCenter* column, let's fix the one row that's missing data and fill it in with the right information.

In [ ]:
df.loc[df['DistanceToCenter'].isna()]

,Suburb,Address,Rooms,Type,Price,SaleMethod,Agency,SellDate,DistanceToCenter,Postcode,Regionname,SellDay,SellMonth,SellYear
14440,Footscray Lot,2/16 Stafford St,2,u,710000,S,Jas,2017-07-15,NaN,NaN,NaN,15,7,2017


In [ ]:
df.loc[df['DistanceToCenter'].isna()] = 3.5

In [ ]:
# Replace the missing postcode
df.loc[df['Postcode'].isna()] = 3218

In [ ]:
# change postcode column type from float to int
df['Postcode'] = df['Postcode'].astype(int)

## **Data Analysis | Real Estate Agencies Performance**

### 1. The agency with the highest sales

In [ ]:
df.Agency.unique()

# let's look both two years 2016 and 2017
tsales_by_agency = df.groupby('Agency')['Price'].sum().reset_index().sort_values('Price',ascending=False).reset_index(drop=True).head(15)
tsales_by_agency.rename(columns={'Price':'Total Sales in $'}, inplace=True)
tsales_by_agency

,Agency,Total Sales in $
0,Jellis,"2,383,028,598"
1,Nelson,"1,992,624,788"
2,Marshall,"1,695,939,138"
3,hockingstuart,"1,485,243,749"
4,Barry,"1,192,982,264"
5,Buxton,"1,140,965,035"
6,Ray,"880,563,562"
7,Fletchers,"520,323,501"
8,Biggin,"493,375,900"
9,Woodards,"425,039,221"


In [ ]:
# let's look at the year 2016
tsales_by_agency = df.query("SellYear == 2016").groupby('Agency')['Price'].sum().reset_index().sort_values('Price',ascending=False).reset_index(drop=True).head(15)
tsales_by_agency.rename(columns={'Price':'Total Sales in $'}, inplace=True)
tsales_by_agency

,Agency,Total Sales in $
0,Jellis,"1,121,942,065"
1,Nelson,"1,004,971,049"
2,Marshall,"813,638,250"
3,hockingstuart,"718,097,289"
4,Buxton,"507,656,685"
5,Barry,"463,936,688"
6,Ray,"259,079,400"
7,Biggin,"246,177,000"
8,Fletchers,"221,713,800"
9,RT,"221,231,000"


In [ ]:
# let's look at the latest year 2017
tsales_by_agency = df.query("SellYear == 2017").groupby('Agency')['Price'].sum().reset_index().sort_values('Price',ascending=False).reset_index(drop=True).head(15)
tsales_by_agency.rename(columns={'Price':'Total Sales in $'}, inplace=True)
tsales_by_agency

,Agency,Total Sales in $
0,Jellis,"1,261,086,533"
1,Nelson,"987,653,739"
2,Marshall,"882,300,888"
3,hockingstuart,"767,146,460"
4,Barry,"729,045,576"
5,Buxton,"633,308,350"
6,Ray,"621,484,162"
7,Fletchers,"298,609,701"
8,Biggin,"247,198,900"
9,Woodards,"243,223,487"


🥇 Jellis looks like the **#1 agency** in Melbourne for two years in a row


---
*timeframe : 2016+2017*


**Let's dive into Jellis Agency and see their monthly performance in the past 2 years**

In [ ]:
jellis = df.query("Agency == 'Jellis'").groupby('SellMonth')['Price'].sum().reset_index().sort_values('SellMonth',ascending=True).reset_index(drop=True)
jellis.rename(columns={'Price':'Total Sales in $'}, inplace=True)
jellis

,SellMonth,Total Sales in $
0,2,"93,851,200"
1,3,"144,713,500"
2,4,"101,049,776"
3,5,"368,894,757"
4,6,"295,054,000"
5,7,"198,427,000"
6,8,"344,792,600"
7,9,"423,297,300"
8,10,"90,743,500"
9,11,"213,539,655"


😞 No sales in January! But September was a good month, selling **±$423M** worth of properties.

In [ ]:
jellis['Total Sales in $'].astype(int)

fig = px.bar(jellis, x='SellMonth', y='Total Sales in $', text_auto='.2s', title="<b>Jellis Sales Performance</b>")
fig.show()

### 2. The agency with most properties sold

In [ ]:
tcount_by_agency = df.groupby('Agency')['Price'].count().reset_index().sort_values('Price',ascending=False).reset_index(drop=True).head(10)
tcount_by_agency.rename(columns={'Price':'Number of Sales'}, inplace=True)
tcount_by_agency

,Agency,Number of Sales
0,Nelson,2002
1,Jellis,1759
2,hockingstuart,1580
3,Barry,1390
4,Ray,1032
5,Buxton,939
6,Marshall,887
7,Biggin,496
8,Brad,442
9,Woodards,411


💎**Nelson** Agency sold more properties than Jellis.

### 3. Who sold the most expensive house?

In [ ]:
df['SellYear'] = df['SellYear'].astype(int)
df.sort_values('Price',ascending=False).reset_index(drop=True).head(10)

,Suburb,Address,Rooms,Type,Price,SaleMethod,Agency,SellDate,DistanceToCenter,Postcode,Regionname,SellDay,SellMonth,SellYear
0,Mulgrave,35 Bevis St,3,h,"9,000,000",PI,Hall,2017-07-29 00:00:00,19,3170,South-Eastern Metropolitan,29,7,2017
1,Canterbury,49 Mangarra Rd,5,h,"8,000,000",VB,Sotheby's,2017-05-13 00:00:00,9,3126,Southern Metropolitan,13,5,2017
2,Hawthorn,49 Lisson Gr,4,h,"7,650,000",S,Abercromby's,2017-06-17 00:00:00,5,3122,Southern Metropolitan,17,6,2017
3,Armadale,42 Hampden Rd,4,h,"6,800,000",VB,Marshall,2017-09-16 00:00:00,6,3143,Southern Metropolitan,16,9,2017
4,Kew,15 Barry St,6,h,"6,500,000",S,Jellis,2016-08-13 00:00:00,6,3101,Southern Metropolitan,13,8,2016
5,Toorak,21 Iona Av,4,h,"6,460,000",S,RT,2017-06-17 00:00:00,4,3142,Southern Metropolitan,17,6,2017
6,Middle Park,136 Page St,5,h,"6,400,000",S,Marshall,2017-09-09 00:00:00,3,3206,Southern Metropolitan,9,9,2017
7,Toorak,17 Albany Rd,3,h,"6,250,000",S,Kay,2016-10-15 00:00:00,5,3142,Southern Metropolitan,15,10,2016
8,Kew,7 Wimba Av,5,h,"6,240,000",S,Jellis,2016-09-03 00:00:00,6,3101,Southern Metropolitan,3,9,2016
9,Armadale,9 Alleyne Av,5,h,"5,925,000",S,Marshall,2017-06-24 00:00:00,6,3143,Southern Metropolitan,24,6,2017


According to this data, HALL agency sold the most expensive property was a house, sold at $9M, in 2017, in July, far from the city at 19 km from the center, with only 3 rooms , with landsize of 744 square meters, in Mulgrave suburb.
 [See property](https://www.realestate.com.au/property/35-bevis-st-mulgrave-vic-3170/)


*However, after investigating this information, I realized it's not true, and we deal with a data quality issue. This property sold under $1M.*

**Sotheby's sold the most expensive property in Canterbury suburb, a house with 5 rooms, for $8M, 9km from the center.** [See property](https://www.realestate.com.au/sold/property-house-vic-canterbury-135352390)



## In which price ranges were the most properties sold?

We need to create a function in Python that segments the prices into more categories:
- Under 1M
- Between 1M - 3M
- Between 3M - 5M
- Between 5M - 8M
- +8M

In [ ]:
# segmentation function

def get_price_range(price):
  if price < 1000000:
    return 'Under 1M'
  elif price >= 1000000 and price <3000000:
    return 'Between 1M - 3M'
  elif price >= 3000000 and price <5000000:
    return 'Between 3M - 5M'
  elif price >= 5000000 and price <8000000:
    return 'Between 5M - 8M'
  else:
    return '+8M'


In [ ]:
# apply function across our price column
df['Price Range'] = df['Price'].apply(get_price_range)

In [ ]:
df.head(5)

,Suburb,Address,Rooms,Type,Price,SaleMethod,Agency,SellDate,DistanceToCenter,Postcode,Regionname,SellDay,SellMonth,SellYear,Price Range
0,Abbotsford,85 Turner St,2,h,"1,480,000",S,Biggin,2016-12-03 00:00:00,2,3067,Northern Metropolitan,3,12,2016,Between 1M - 3M
1,Abbotsford,25 Bloomburg St,2,h,"1,035,000",S,Biggin,2016-02-04 00:00:00,2,3067,Northern Metropolitan,4,2,2016,Between 1M - 3M
2,Abbotsford,5 Charles St,3,h,"1,465,000",SP,Biggin,2017-03-04 00:00:00,2,3067,Northern Metropolitan,4,3,2017,Between 1M - 3M
3,Abbotsford,40 Federation La,3,h,"850,000",PI,Biggin,2017-03-04 00:00:00,2,3067,Northern Metropolitan,4,3,2017,Under 1M
4,Abbotsford,55a Park St,4,h,"1,600,000",VB,Nelson,2016-06-04 00:00:00,2,3067,Northern Metropolitan,4,6,2016,Between 1M - 3M


In [ ]:
tcount_price_range = df.query("Agency == 'Jellis'").groupby('Price Range')['Suburb'].count().reset_index().reset_index(drop=True).head(10)
tcount_price_range.rename(columns={'Suburb':'Number of Sales'}, inplace=True)
tcount_price_range

,Price Range,Number of Sales
0,Between 1M - 3M,1060
1,Between 3M - 5M,66
2,Between 5M - 8M,5
3,Under 1M,628


We can conclude that Jellis excelled in selling properties under 3M. 🏠